# <font color=#f4665a>  ICA analysis </font>

### Project: BMPD HC
____________________________________________________

**Description:** This notebook provides code for BOLD signal fMRI resting-state processing for the Biomarker for Parkinson's Disease (BMPD)data. 
We will used ICA analysis:
CanICA is an ICA package for group-level analysis of fMRI data.  
It brings a well-controlled group model, as well as a thresholding algorithm controlling for specificity and sensitivity with an explicit model of the signal.  
The reference papers are: G. Varoquaux et al. "A group model for stable multi-subject ICA on fMRI datasets", NeuroImage Vol 51 (2010), p. 288-299
G. Varoquaux et al. "ICA-based sparse features recovery from fMRI datasets", IEEE ISBI 2010, p. 1177


**Toolbox required:** SpinalCordToolbox, FSL, nilearn toolbox, nipype, matlab

**Inputs**:  
This notebook required this the following prepross anatomical,fmri images 

**Ouputs**:
See the output description at each step of the Notebook.

____________________________________________________


## <font color=#00988c> Imports </font>

In [1]:
import sys
import json
# Spinal cord Toolbox_________________________________________
### Cerebro:
sys.path.append("/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/spinalcordtoolbox-5.0.0")
sys.path.append("/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/spinalcordtoolbox-5.0.0/scripts") #sys.path.insert(0, "/cerebro/cerebro1/dataset/bmpd/derivatives/sc_preproc/code/sct/spinalcordtoolbox")
sys.path.append("/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/hc_project_analyses/code/") #sys.path.insert(0, "/cerebro/cerebro1/dataset/bmpd/derivatives/sc_preproc/code/sct/spinalcordtoolbox")

from spinalcordtoolbox.utils.sys import run_proc
import glob
from nilearn.maskers import NiftiMasker


from canICA_analyses import ICA
%matplotlib inline
%load_ext autoreload
%autoreload 2


/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/bmpd_python/lib/python3.6/site-packages/nilearn/__init__.py:69: FutureWarning: Python 3.6 support is deprecated and will be removed in release 0.10 of Nilearn. Consider switching to Python 3.8 or 3.9.
  _python_deprecation_warnings()


## <font color=#00988c>  I. Run the canICA analysis </font>

In [38]:
# Load the dataset config
#config_spine_only_CL.json #../config/config_brsc_CL.json
with open('../config/config_spine_only_CL.json') as config_file:
    config = json.load(config_file)
dataset="gva" 
structures=["spinalcord"] # ["spinalcord"] or ["brain","spinalcord"] . double check the script for brainsc
config["data"][dataset]["ica"]["spinalcord"]["dir"]='/ICA/results_spine_only/gva/spinalcord/temporal_cropping/9min/'
config["data"][dataset]['inputs_ica']['spinalcord']['tag_filename_spinalcord']='moco_HP_sc_inTemplate_216vol_s.nii.gz'

In [39]:
files_func={};func_allsbj={}
for structure in structures:
    if len(structures) == 1:
        ana=structure
    else:
        ana= "brain_spinalcord"
    files_func[structure]=[];func_allsbj[structure]=[]
    for sbj_nb in range(len(config["list_subjects"][dataset])):
        subject_name=config["list_subjects"][dataset][sbj_nb]
        files_func[structure].append(glob.glob(config["data"][dataset]["inputs_ica"]["dir"]+ '/sub-' + subject_name + '/'  + structure + '/*' + config["data"][dataset]["inputs_ica"][structure]["tag_filename_" + ana] + '*')[0])
print(files_func[structure])    

['/cerebro/cerebro1/dataset/caroline/2022_SPiCiCAP_NK_CL/CL_analyses/denoising/3_func_smoothed//sub-01/spinalcord/sub-01_fmri_moco_HP_sc_inTemplate_216vol_s.nii.gz', '/cerebro/cerebro1/dataset/caroline/2022_SPiCiCAP_NK_CL/CL_analyses/denoising/3_func_smoothed//sub-02/spinalcord/sub-02_fmri_moco_HP_sc_inTemplate_216vol_s.nii.gz', '/cerebro/cerebro1/dataset/caroline/2022_SPiCiCAP_NK_CL/CL_analyses/denoising/3_func_smoothed//sub-03/spinalcord/sub-03_fmri_moco_HP_sc_inTemplate_216vol_s.nii.gz', '/cerebro/cerebro1/dataset/caroline/2022_SPiCiCAP_NK_CL/CL_analyses/denoising/3_func_smoothed//sub-04/spinalcord/sub-04_fmri_moco_HP_sc_inTemplate_216vol_s.nii.gz', '/cerebro/cerebro1/dataset/caroline/2022_SPiCiCAP_NK_CL/CL_analyses/denoising/3_func_smoothed//sub-05/spinalcord/sub-05_fmri_moco_HP_sc_inTemplate_216vol_s.nii.gz', '/cerebro/cerebro1/dataset/caroline/2022_SPiCiCAP_NK_CL/CL_analyses/denoising/3_func_smoothed//sub-06/spinalcord/sub-06_fmri_moco_HP_sc_inTemplate_216vol_s.nii.gz', '/cerebro

In [40]:
redo=True
config["ica_ana"]["k_range"]["spinalcord"]=[5]
for k in config["ica_ana"]["k_range"]["spinalcord"]:
    config["ica_ana"]["n_comp"]=k # usefull if you want to test only on k
    print(config["ica_ana"]["iter"])
    
    icas = ICA(files_func[structure],[''],structures,dataset,config) # "brain_spinalcord" or "brain" or "spinalcord"
    #icas = ICA(files_func[structures[0]],files_func[structures[1]],structures,dataset,config) # "brain_spinalcord" or "brain" or "spinalcord"
    if k==4:
        all_data=icas.get_data(run='extract',t_r=config["acq_params"][dataset]["TR"],n_jobs=1) # load or extract
    if redo==True:
        all_data=icas.get_data(run='extract',t_r=config["acq_params"][dataset]["TR"],n_jobs=8) # load or extract, if NaN issues put both
        all_data=icas.get_data(run='load',t_r=config["acq_params"][dataset]["TR"],n_jobs=8) # load or extract, if NaN issues put both
        reducedata_all=icas.indiv_PCA(all_data,save_indiv_img=True) # that step is not implanted to save individual maps for brain + sc yet
        components=icas.get_CCA(reducedata_all)
        components_final,components_final_z=icas.get_ICA(components)
        zcomponents4D_filename=icas.save_components(components_final,components_final_z)
    
    

500
Analyse will be run on spinalcord structure
Mask: /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//templates/PAM50/template/PAM50_cord_C5toC8.nii.gz
- 4D image create for sbj  01
- 4D image create for sbj  02
- 4D image create for sbj  03
- 4D image create for sbj  04
- 4D image create for sbj  05
- 4D image create for sbj  06
- 4D image create for sbj  07
- 4D image create for sbj  08
- 4D image create for sbj  09
- 4D image create for sbj  10
- 4D image create for sbj  12
- 4D image create for sbj  11
- 4D image create for sbj  13
- 4D image create for sbj  14
- 4D image create for sbj  15
- 4D image create for sbj  16
- 4D image create for sbj  17
- 4D image create for sbj  18
- 4D image create for sbj  19
>> Individual PCA done <<


/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/bmpd_python/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/bmpd_python/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/bmpd_python/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/bmpd_python/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120:

>> Group ICA done <<
>> Components z-scored done <<


In [45]:
reducedata_all=icas.indiv_PCA(all_data,save_indiv_img=True) # that step is not implanted to save individual maps for brain + sc yet
components=icas.get_CCA(reducedata_all)
components_final,components_final_z=icas.get_ICA(components)
zcomponents4D_filename=icas.save_components(components_final,components_final_z)
    

- 4D image create for sbj  01
- 4D image create for sbj  02
- 4D image create for sbj  03
- 4D image create for sbj  04
- 4D image create for sbj  05
- 4D image create for sbj  06
- 4D image create for sbj  07
- 4D image create for sbj  08
- 4D image create for sbj  09
- 4D image create for sbj  10
- 4D image create for sbj  12
- 4D image create for sbj  11
- 4D image create for sbj  13
- 4D image create for sbj  14
- 4D image create for sbj  15
- 4D image create for sbj  16
- 4D image create for sbj  17
- 4D image create for sbj  18
- 4D image create for sbj  19
>> Individual PCA done <<


/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/bmpd_python/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/bmpd_python/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/bmpd_python/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/bmpd_python/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120:

>> Group ICA done <<
>> Components z-scored done <<


## <font color=#00988c> II.  Run individual ICA </font>

In [ ]:
import os 
outputdir=[]
config["ica_ana"]["iter"]=500
for sbj_nb in range(0,1):#len(config["list_subjects"][dataset])):
    subject_name=config["list_subjects"][dataset][sbj_nb]
    print(subject_name)
    config["ica_ana"]["n_comp"]=9
    
    icas = ICA([files_func[structure][sbj_nb]],[''],structures,dataset,config,subject_name) 
    
    # extract individual data
    components=icas.get_data(run='extract',t_r=config["acq_params"][dataset]["TR"],n_jobs=8) # load or extract, if NaN issues put both
    components_final,components_final_z=icas.get_ICA(components.T,k=9) # components (n_voxels,n_volumes)
    zcomponents4D_filename=icas.save_components(components_final,components_final_z,one_subject=subject_name)
    

In [48]:
config["ica_ana"]["iter"]=5

components_final,components_final_z=icas.get_ICA(components.T,k=9)

/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/bmpd_python/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/bmpd_python/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/bmpd_python/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  ConvergenceWarning)
/cerebro/cerebro1/dataset/bmpd/derivatives/thibault_test/code/toolbox/bmpd_python/lib/python3.6/site-packages/sklearn/decomposition/_fastica.py:120:

>> Group ICA done <<


In [50]:
zcomponents4D_filename=icas.save_components(components_final,components_final_z)
   

/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project//ICA/results_spine_only/mtl/spinalcord////K_9/comp_indiv/CanICA_sub-P033_spinalcord_spinalcord_4D_K_9.nii.gz
>> Components z-scored done <<


## <font color=#00988c> III.  Analyse to test iteration </font>

In [ ]:
redo=False
for iter in [2000,5000]:
    config["data"][dataset]['ica']["spinalcord"]['dir']='/ICA/results_spine_only/mtl/spinalcord/iterations_tests/' + str(iter) + "/"
    config['ica_ana']['iter']=iter
    config["ica_ana"]["k_range"]["spinalcord"]=[9]
    print("Analyse is running for " + dataset + "and k= " + str(config["ica_ana"]["k_range"]["spinalcord"]) + " iter: " + str(iter))
    for k in config["ica_ana"]["k_range"]["spinalcord"]:
        config["ica_ana"]["n_comp"]=k # usefull if you want to test only on k
        print(config["ica_ana"]["n_comp"])
        icas = ICA(files_func[structure],[''],structures,dataset,config) # "brain_spinalcord" or "brain" or "spinalcord"
        
        if k==4:
            all_data=icas.get_data(run='extract',t_r=config["acq_params"][dataset]["TR"],n_jobs=1) # load or extract
        if redo==True:
            all_data=icas.get_data(run='extract',t_r=config["acq_params"][dataset]["TR"],n_jobs=8) # load or extract, if NaN issues put both
            all_data=icas.get_data(run='load',t_r=config["acq_params"][dataset]["TR"],n_jobs=8) # load or extract, if NaN issues put both
            reducedata_all=icas.indiv_PCA(all_data,save_indiv_img=True) # that step is not implanted to save individual maps for brain + sc yet
            components=icas.get_CCA(reducedata_all)
            components_final,components_final_z=icas.get_ICA(components)
            zcomponents4D_filename=icas.save_components(components_final,components_final_z)

## <font color=#00988c>  IV. Run spatial regression </font>
To obtain matrices describing temporal dynamics for each component and subject

In [ ]:
# transform image into array
load_img=True
if load_img==True:
    comp_raw_dir= config["main_dir"] + config["data"][dataset]["ica"][structure]["dir"] + '/' + '/K_' + str(config["ica_ana"]["n_comp"]) + '/comp_raw/'
    zcomponents4D_filename= glob.glob(comp_raw_dir + "*4D*")[0]
    nifti_masker= NiftiMasker(mask_img=config["main_dir"] + config["masks"][dataset][structure]).fit() #Extract the data inside the mask and create a vector
    components_final=nifti_masker.transform(zcomponents4D_filename).T # array for one subject, shape: n_volumes,n_voxels
else:
    components_final=components_final
    
output_dir=config["main_dir"] + config["data"][dataset]["ica"][structure]["dir"] + '/' + '/K_' + str(config["ica_ana"]["n_comp"]) + '/comp_indiv_dynamic/'
if not os.path.exists(output_dir):
    os.mkdir(output_dir)  

In [ ]:
temporal_dynamics_txt=[]
temporal_dynamics=[]
for sbj_nb in range(len(config["list_subjects"][dataset])):
    subject_name=config["list_subjects"][dataset][sbj_nb]
    temporal_dynamics_txt.append(output_dir + "sub-" + subject_name + "_K_" + str(config["ica_ana"]["n_comp"]) + "_temporal_dynamic.txt")
    temporal_dynamics.append(icas.spatial_regression(components_final,all_data[sbj_nb],save=True,output_filename=temporal_dynamics_txt[sbj_nb]))

In [ ]:
import glob
dataset="mtl"
func=[]
root_prep="/cerebro/cerebro1/dataset/"
for sbj_nb in range(len(config["list_subjects"][dataset])):
    subject_name=config["list_subjects"][dataset][sbj_nb]
    if sbj_nb <12:
        datas="bmpd/derivatives/spinalcord_processing/"
    else:
        datas="stratals/derivatives/preprocessing/"
    func.append(glob.glob(root_prep + datas + "sub-" + subject_name + "/func/5_Coregistration/spinalcord/*moco_mean_coreg_in_PAM50.nii.gz")[0])

In [11]:
from nilearn import image

In [35]:
new_4D=image.concat_imgs(func)

In [36]:
func_4D="/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/templates/PAM50/template/func_group_4D.nii.gz"
new_4D.to_filename(func_4D)

In [38]:
func_mean="/cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/templates/PAM50/template/func_group_mean.nii.gz"
run_proc("fslmaths {} -Tmean {}".format(func_4D,func_mean))

fslmaths /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/templates/PAM50/template/func_group_4D.nii.gz -Tmean /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/templates/PAM50/template/func_group_mean.nii.gz # in /cerebro/cerebro1/dataset/bmpd/derivatives/HealthyControls_project/hc_project_analyses/notebook


(0, '')

In [ ]:
range(len(config["list_subjects"][dataset]))